In [1]:
import warnings

warnings.filterwarnings("ignore")

import copy
import json
import os
import re
import sys
from collections import Counter
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rich import print
from scipy.ndimage import convolve
from tqdm import tqdm

In [2]:
def read_file_to_str_li(fp, print_exp=True):
    with open(fp, "r") as f:
        lines = f.read().split("\n")
    if print_exp:
        print(f"Read from {fp}:")
        print(f"First line: {lines[0]} | Last line: {lines[-1]}")
        print("-" * 6)

    return lines


# define the function blocks
def convert_to_int(input_str):
    if input_str == "" or input_str == " ":
        return None
    return int(input_str)


def convert_to_str(input_str):
    return str(input_str)


# map the inputs to the function blocks
converts = {
    "i": convert_to_int,
    "s": convert_to_str,
}


def convert_str_li_to_other_li(
    str_li, pattern="i", per_letter=False, sep=" ", start_row=0, end_row=None
):
    """ Convert a list of string to a list of other types
    
    pattern: a list of types for one item. 
        'i' for int, 's' for string
        'si' means: convert the 1st item to string, the rest to integer
        If separated items are more than pattern items,
        use the last one from the parttern.
    if per_letter=True, ignore sep and separate item per letter
    """
    target_str_li = str_li[start_row:end_row]
    # find max item num
    max_item_num = 1
    if per_letter:
        max_item_num = max([len(s) for s in target_str_li])
    else:
        max_item_num = max([len(s.split(sep)) for s in target_str_li])

    # extend the pattern to the max itme num
    pattern = (
        pattern + f"{pattern[-1]}" * (max_item_num - len(pattern))
        if max_item_num > len(pattern)
        else pattern
    )

    # convert
    if per_letter:
        return [
            [converts[pattern[idx]](item) for idx, item in enumerate(s)]
            for s in target_str_li
        ]
    else:
        if sep == " ":
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split())]
                for s in target_str_li
            ]
        else:
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split(sep))]
                for s in target_str_li
            ]

In [3]:
fp = "input.txt"
lines = read_file_to_str_li(fp)

print("Convert to:")

head = convert_str_li_to_other_li(
    lines, pattern="s", per_letter=True, sep=",", start_row=0, end_row=1
)

print(f"Head:\n{head}")
# print(f"First line: {head[0]}")
# print(f"Last line: {head[-1]}")

# data = convert_str_li_to_other_li(
#     lines, pattern="s", per_letter=True, sep=" ", start_row=None, end_row=None
# )
data = convert_str_li_to_other_li(
    lines, pattern="s", per_letter=True, sep=",", start_row=2, end_row=None
)
# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=False, sep=",", start_row=None, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="s", per_letter=False, sep=" -> ", start_row=None, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=False, sep=" ", start_row=2, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="si", per_letter=False, sep=" ", start_row=0, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=True, sep=" ", start_row=0, end_row=None
# )

print(f"First line: {data[0]}")
print(f"Last line: {data[-1]}")
print("-" * 6)

Read from input.txt:

First line: ########...#.###.##..#.....#####....#.#...#..##..##.###.....####...#.#..#.###...#
.#.##...#..##.###..##......##....##....#.#..#...##.###..#.#.#####..##.##....#.#####.#.######.
..#.####....#.#.####..#.####.#.#..##......#....#.##...#...#...#......#...#...#.#.....###..#.#
#..#...####.##....#.#.#...##...####.#####.####.###.##.###.##...#.#.#...###.######....#.#.#.#.
.#.#....#####.#.##...##...##...##...#.#.#####.#...#.#######..#...##..#..#...#..####....###..#
..##.##..#.#.##...#####......#.#.###.####...#.#.###.#.##.#. | Last line: ..##...#.#.#.####.#.
.#.####...#...###.#..###..#.###.####....#.#....#...##...#.#.#....##.#.#.##....##

------

Convert to:

Head:
[['#', '#', '#', '#', '#', '#', '#', '#', '.', '.', '.', '#', '.', '#', '#', '#', '.', '#', 
'#', '.', '.', '#', '.', '.', '.', '.', '.', '#', '#', '#', '#', '#', '.', '.', '.', '.', 
'#', '.', '#', '.', '.', '.', '#', '.', '.', '#', '#', '.', '.', '#', '#', '.', '#', '#', 
'#', '.', '.', '.', '.', '.', '#', '#', '#', '#', '.', '.', '.', '#', '.', '#', '.', '.', 
'#', '.', '#', '#', '#', '.', '.', '.', '#', '.', '#', '.', '#', '#', '.', '.', '.', '#', 
'.', '.', '#', '#', '.', '#', '#', '#', '.', '.', '#', '#', '.', '.', '.', '.', '.', '.', 
'#', '#', '.', '.', '.', '.', '#', '#', '.', '.', '.', '.', '#', '.', '#', '.', '.', '#', 
'.', '.', '.', '#', '#', '.', '#', '#', '#', '.', '.', '#', '.', '#', '.', '#', '#', '#', 
'#', '#', '.', '.', '#', '#', '.', '#', '#', '.', '.', '.', '.', '#', '.', '#', '#', '#', 
'#', '#', '.', '#', '.', '#', '#', '#', '#', '#', '#', '.', '.', '.', '#', '.', '#', '#', 
'#', '#', '.', '.', '.', '.', '#', '.', '#', '.', '#', '#', '#', '#', '.', '.', '#', '.', 
'#', '#', '#', '#', '.', '#', '.', '#', '.', '.', '#', '#', '.', '.', '.', '.', '.', '.', 
'#', '.', '.', '.', '.', '#', '.', '#', '#', '.', '.', '.', '#', '.', '.', '.', '#', '.', 
'.', '.', '#', '.', '.', '.', '.', '.', '.', '#', '.', '.', '.', '#', '.', '.', '.', '#', 
'.', '#', '.', '.', '.', '.', '.', '#', '#', '#', '.', '.', '#', '.', '#', '#', '.', '.', 
'#', '.', '.', '.', '#', '#', '#', '#', '.', '#', '#', '.', '.', '.', '.', '#', '.', '#', 
'.', '#', '.', '.', '.', '#', '#', '.', '.', '.', '#', '#', '#', '#', '.', '#', '#', '#', 
'#', '#', '.', '#', '#', '#', '#', '.', '#', '#', '#', '.', '#', '#', '.', '#', '#', '#', 
'.', '#', '#', '.', '.', '.', '#', '.', '#', '.', '#', '.', '.', '.', '#', '#', '#', '.', 
'#', '#', '#', '#', '#', '#', '.', '.', '.', '.', '#', '.', '#', '.', '#', '.', '#', '.', 
'.', '#', '.', '#', '.', '.', '.', '.', '#', '#', '#', '#', '#', '.', '#', '.', '#', '#', 
'.', '.', '.', '#', '#', '.', '.', '.', '#', '#', '.', '.', '.', '#', '#', '.', '.', '.', 
'#', '.', '#', '.', '#', '#', '#', '#', '#', '.', '#', '.', '.', '.', '#', '.', '#', '#', 
'#', '#', '#', '#', '#', '.', '.', '#', '.', '.', '.', '#', '#', '.', '.', '#', '.', '.', 
'#', '.', '.', '.', '#', '.', '.', '#', '#', '#', '#', '.', '.', '.', '.', '#', '#', '#', 
'.', '.', '#', '.', '.', '#', '#', '.', '#', '#', '.', '.', '#', '.', '#', '.', '#', '#', 
'.', '.', '.', '#', '#', '#', '#', '#', '.', '.', '.', '.', '.', '.', '#', '.', '#', '.', 
'#', '#', '#', '.', '#', '#', '#', '#', '.', '.', '.', '#', '.', '#', '.', '#', '#', '#', 
'.', '#', '.', '#', '#', '.', '#', '.']]

First line: ['#', '#', '#', '.', '.', '.', '.', '#', '#', '#', '.', '#', '.', '#', '#', '#', 
'#', '#', '.', '#', '.', '.', '#', '.', '.', '#', '.', '#', '#', '.', '.', '.', '#', '#', 
'.', '#', '.', '#', '#', '.', '#', '.', '.', '#', '#', '.', '.', '#', '#', '#', '#', '.', 
'.', '#', '#', '.', '.', '.', '.', '#', '.', '#', '#', '.', '#', '#', '#', '.', '.', '#', 
'#', '.', '.', '.', '.', '#', '#', '#', '#', '#', '.', '#', '.', '.', '#', '.', '#', '#', 
'#', '#', '#', '#', '.', '#', '.', '#', '.', '#', '#', '.']

Last line: ['.', '.', '#', '#', '.', '.', '.', '#', '.', '#', '.', '#', '.', '#', '#', '#', 
'#', '.', '#', '.', '.', '#', '.', '#', '#', '#', '#', '.', '.', '.', '#', '.', '.', '.', 
'#', '#', '#', '.', '#', '.', '.', '#', '#', '#', '.', '.', '#', '.', '#', '#', '#', '.', 
'#', '#', '#', '#', '.', '.', '.', '.', '#', '.', '#', '.', '.', '.', '.', '#', '.', '.', 
'.', '#', '#', '.', '.', '.', '#', '.', '#', '.', '#', '.', '.', '.', '.', '#', '#', '.', 
'#', '.', '#', '.', '#', '#', '.', '.', '.', '.', '#', '#']

------

In [4]:
def update_img(img, pad_value=0):
    """
    pad_value is very important
    at the beginning the external lights are off
    but since the enh_map[0]=1, after one update, all the external light are ON !!
    so the padding value should be changed to 1, then back to 0, and so on
    """
    img = np.pad(img, 3, "constant", constant_values=pad_value)
    img = convolve(img, kernal, mode="constant", cval=pad_value)
    img = np.vectorize(enh_map.get)(img)
    return img, img[0, 0]


def update_n_times(img, n):
    img = init_img
    pad_value = 0
    for _ in range(n):
        img, pad_value = update_img(img, pad_value)
    return np.sum(img)


enh_map = {idx: 1 if v == "#" else 0 for idx, v in enumerate(head[0])}
init_img = np.where(np.array(data) == "#", 1, 0)
kernal = np.array([[1, 2, 4], [8, 16, 32], [64, 128, 256]])

print(f"Answer to Q1: {update_n_times(init_img, 2)}")
print(f"Answer to Q2: {update_n_times(init_img, 50)}")

Answer to Q1: 5361

Answer to Q2: 16826